In [ ]:
print(os.getcwd())
dir=os.getcwd()
os.system("pip install gitpython")
import git  # pip install gitpython
g = git.cmd.Git(dir)
g.pull()

ModuleNotFoundError: No module named 'git'

In [ ]:
from qick import *
from qick.asm_v2 import *
%matplotlib inline
import matplotlib.pyplot as plt
from os import *

#This line is to sync to an external clock which needs to be 10 Mhz
#soc = QickSoc(external_clk=True)
soc = QickSoc()
soc = QickSoc(bitfile ='../qick_lib/qick/qick_216_rfbv2.bit', clk_output=True)
soccfg = soc
print(soccfg)


/home/xilinx/jupyter_notebooks/amo_qick/tests


OSError: Bitstream file ../qick_lib/qick/qick_216_rfbv2.bit does not exist.

In [ ]:
GEN_CH=0 # DAC 
RO_CH=0  # ADC
#BASE WORKABILITY: PLEASE DO NOT CHANGE 
class LoopbackProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        self.declare_gen(ch=gen_ch, nqz=cfg['nqz'], mixer_freq=cfg['mixer_freq'], ro_ch=ro_ch)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])
        self.add_readoutconfig(ch=ro_ch, name="myro",
                               freq=cfg['freq'],
                               gen_ch=gen_ch,
                               outsel='product')
        self.add_cosine(ch=gen_ch, name="ramp", length=cfg['ramp_len'], even_length=True)
        self.add_pulse(ch=gen_ch, name="mypulse", ro_ch=ro_ch,
                       style="const",
#                        style="flat_top", 
#                        envelope="ramp", 
                       freq=cfg['freq'],
                       length=cfg['flat_len'],
                       phase=cfg['phase'],
                       gain=cfg['gain'],
                      )
        self.send_readoutconfig(ch=cfg['ro_ch'], name="myro", t=0)
        
        
    def _body(self, cfg):
        self.delay_auto()
        self.pulse(ch=cfg['gen_ch'], name="mypulse", t=0.0)
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'], mr=True)
        
config = {'gen_ch': GEN_CH,
          'ro_ch': RO_CH,
          'mixer_freq': 100,
          'freq': 10,
          'nqz': 1,
          'trig_time': 0.4,
          'ro_len': .1,
          'flat_len': .2,
          'ramp_len': 0.01,
          'phase': 0,
          'gain': 1.0
         }

prog = LoopbackProgram(soccfg, reps=1, final_delay=0.5, cfg=config)
freq = config['freq']

#external trigger is currently available for tproc_2, on io pin 5, remove start_src if you do not want the trigger

iq_list = prog.acquire_decimated(soc, progress=True, start_src='external')
t = prog.get_time_axis(ro_index=0)
iq = iq_list[0]
plt.plot(t, iq[:,0], label="I value")
plt.plot(t, iq[:,1], label="Q value")
plt.legend()
plt.ylabel("amplitude [ADU]")
plt.xlabel("time [us]");

In [ ]:
# attempting to loop a program for several cycles to achieve a continuous signal:
GEN_CH=0 # DAC 
RO_CH=1  # ADC

class LoopbackProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        self.declare_gen(ch=gen_ch, nqz=cfg['nqz'], mixer_freq=cfg['mixer_freq'], ro_ch=ro_ch)
        self.add_cosine(ch=gen_ch, name="ramp", length=cfg['ramp_len'], even_length=True)
        self.add_pulse(ch=gen_ch, name="mypulse", ro_ch=ro_ch,
                       style="const",
#                        style="flat_top", 
#                        envelope="ramp", 
                       freq=cfg['freq'],
                       length=cfg['flat_len'],
                       phase=cfg['phase'],
                       gain=cfg['gain'],
                      )
        self.add_loop("loopedy_loop", 10000000, exec_before=None, exec_after=None)
        
        
    def _body(self, cfg):
        self.pulse(ch=cfg['gen_ch'], name="mypulse", t=0.0)
        #self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'], mr=True)


        
config = {'gen_ch': GEN_CH,
          'ro_ch': RO_CH,
          'mixer_freq': 100,
          'freq': 100,
          'nqz': 1,
          'trig_time': 0.4,
          'ro_len': .1,
          'flat_len': 1,
          'ramp_len': 0.01,
          'phase': 0,
          'gain': 1.0
         }

prog = LoopbackProgram(soccfg, reps=1, final_delay=0.0, cfg=config)
print(prog)
freq = config['freq']

#external trigger is currently available for tproc_2, on io pin 5, remove start_src if you do not want the trigger
prog.run(soc, start_src='external', progress=True)
#iq_list = prog.acquire_decimated(soc, progress=False, start_src='external')
# t = prog.get_time_axis(ro_index=0)
# iq = iq_list[0]
# plt.plot(t, iq[:,0], label="I value")
# plt.plot(t, iq[:,1], label="Q value")
# plt.legend()
# plt.ylabel("amplitude [ADU]")
# plt.xlabel("time [us]");

In [ ]:
# attempting to sweep a program for several cycles and achieve a continuous signal:
GEN_CH=0 # DAC 
RO_CH=1  # ADC

class SweepProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        frequency = QickSweep1D("loopedy_loop", 100, 200)    ####HERE
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        self.declare_gen(ch=gen_ch, nqz=cfg['nqz'], mixer_freq=cfg['mixer_freq'], ro_ch=ro_ch)
        self.add_pulse(ch=gen_ch, name="mypulse", ro_ch=ro_ch,
                       style="const",
#                        style="flat_top", 
#                        envelope="ramp", 
                       freq=frequency,                      ###HERE
                       length=cfg['flat_len'],
                       phase=cfg['phase'],
                       gain=cfg['gain'],
                      )
        self.add_loop("loopedy_loop", 100000, exec_before=None, exec_after=None)
        
        
    def _body(self, cfg):
        self.pulse(ch=cfg['gen_ch'], name="mypulse", t = 0)
        #self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'], mr=True)


        
config = {'gen_ch': GEN_CH,
          'ro_ch': RO_CH,
          'mixer_freq': 100,
          'freq': 100,
          'nqz': 1,
          'trig_time': 0.4,
          'ro_len': .1,
          'flat_len': 1,
          'ramp_len': 0.01,
          'phase': 0,
          'gain': 1.0
         }

prog = SweepProgram(soccfg, reps=1, final_delay=0.0, cfg=config)
print(prog)
freq = config['freq']

#external trigger is currently available for tproc_2, on io pin 5, remove start_src if you do not want the trigger
prog.acquire_decimated(soc, progress=True, start_src='external')

In [ ]:
GEN_CH=0 # DAC 
RO_CH=10  # ADC
class LoopbackProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        self.declare_gen(ch=gen_ch, nqz=cfg['nqz'], ro_ch=ro_ch)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])
        self.add_readoutconfig(ch=ro_ch, name="myro",
                               freq=cfg['adc_freq'],
                               gen_ch=gen_ch,
                               outsel='product')
        self.add_pulse(ch=gen_ch, name="mypulse", ro_ch=ro_ch,
                       style="const",
#                        style="flat_top", 
#                        envelope="ramp", 
                       freq=cfg['dac_freq'],
                       length=cfg['flat_len'],
                       phase=cfg['phase'],
                       gain=cfg['gain'],
                      )
        self.send_readoutconfig(ch=cfg['ro_ch'], name="myro", t=0)
        
        
    def _body(self, cfg):
        self.delay_auto()
        self.pulse(ch=cfg['gen_ch'], name="mypulse", t=0.0)
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'])
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['ro_len']+cfg['trig_time']+cfg['down_time'])

        
        
        
config = {'gen_ch': GEN_CH,
          'ro_ch': RO_CH,
          'dac_freq': 120,
          'adc_freq': 110,
          'nqz': 1,
          'trig_time': 0.5,
          'ro_len': 0.004,
          'down_time':0.0059,
          'flat_len': .4,
          'ramp_len': 0.01,
          'phase': 0,
          'gain': 1.0
         }

prog = LoopbackProgram(soccfg, reps=1, final_delay=0.5, cfg=config)


#external trigger is currently available for tproc_2, on io pin 5, remove start_src if you do not want the trigger
exp_pnts = prog.acquire(soc, progress=True)
rounds = prog.get_rounds()
t = prog.get_time_axis(ro_index=0)




In [ ]:
avg_arctan = (np.arctan2(-6, 1) - np.arctan2(-1, 1))/2
arctann = np.arctan2(-1,1)
print(avg_arctan, arctann)
def fast_atan(a, b=1):
    if b == 0:
        return 1.57079633
    else:
        x= a/b
        a = [0.994766756708199, -2.8543851807526100E-01, 0.0760699247645105]
        xx = x * x;
        return ((a[2] * xx + a[1]) * xx + a[0]) * x;


def phase_direction():
    self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'])
    self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['ro_len']+cfg['trig_time']+cfg['down_time'])
    

In [ ]:
x_coordinates = np.linspace(-math.pi/2, math.pi/2, 500)
fast_arctan = np.zeros(len(x_coordinates))
for i, value in enumerate(x_coordinates):
    fast_arctan[i]=fast_atan(value)
normal_arctan = np.arctan2(x_coordinates, 1)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(x_coordinates, fast_arctan, label='Fast atan approximation', linestyle='--')
plt.plot(x_coordinates, normal_arctan, label='np.arctan2(x, 1)', linestyle='-')
plt.xlabel('x')
plt.ylabel('arctan value')
plt.title('Comparison of fast atan approximation and numpy arctan2')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

min_val, max_val = -3000, 3000
min_idx_i, max_idx_i = 6000, 0
min_idx_q, max_idx_q = 6000, 0
grid_size = max_val - min_val + 1

In [ ]:
heatmap_data = np.zeros((grid_size, grid_size), dtype=np.float64)
heatmap_data_fast = np.zeros((grid_size, grid_size), dtype=np.float64)
GEN_CH=0 # DAC 
RO_CH=10  # ADC
class LoopbackProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        phase = QickSweep1D("loopedy_loop", 0, 10)    ####HERE

        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        self.declare_gen(ch=gen_ch, nqz=cfg['nqz'])
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])
        self.add_readoutconfig(ch=ro_ch, name="myro",
                               freq=cfg['adc_freq'],
                               gen_ch=gen_ch,
                               outsel='product')
        self.add_pulse(ch=gen_ch, name="mypulse", ro_ch=ro_ch,
                       style="const",
#                        style="flat_top", 
#                        envelope="ramp", 
                       freq=cfg['dac_freq'],
                       length=cfg['flat_len'],
                       phase=phase,
                       gain=cfg['gain'],
                      )
        self.send_readoutconfig(ch=cfg['ro_ch'], name="myro", t=0)
        self.add_loop("loopedy_loop", 1000, exec_before=None, exec_after=None)



    def _body(self, cfg):
        self.delay_auto()
        self.pulse(ch=cfg['gen_ch'], name="mypulse", t=0.0)
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'])
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['ro_len']+cfg['trig_time']+cfg['down_time'])




config = {'gen_ch': GEN_CH,
          'ro_ch': RO_CH,
          'dac_freq': 110,
          'adc_freq': 110,
          'nqz': 1,
          'trig_time': 0.5,
          'ro_len': 0.004,
          'down_time':0.0059,
          'flat_len': .4,
          'ramp_len': 0.01,
          'phase': .875,
          'gain': 1.0
         }

prog = LoopbackProgram(soccfg, reps=1, final_delay = 0.1, cfg=config)


#external trigger is currently available for tproc_2, on io pin 5, remove start_src if you do not want the trigger
exp_pnts = prog.acquire(soc, progress = False)
rounds = prog.get_rounds()
t = prog.get_time_axis(ro_index=0)
data = rounds[0][0][0]
x = np.linspace(0,10,len(data))
y = data[:, 0]
z = data[:, 1]
plt.figure()
plt.plot(x, y, label='y data')
plt.title('Trend lines for IQ data')
plt.ylabel('I Values (Arbitrary)')

plt.figure()
plt.plot(x, z)
plt.ylabel('Q Values (Arbitrary)')

plt.figure()
plt.scatter(x[1:], np.diff(np.arctan2(z,y)), s=5)
plt.xlabel('$\phi$ (Arbitrary)')
plt.ylabel('$\Delta\phi (radians)$' )
plt.show()


In [ ]:
valid_points = ~np.isnan(heatmap_data)
y_coords, x_coords = np.where(valid_points)
values = heatmap_data[valid_points]

plt.figure(figsize=(12,10))
scatter = plt.scatter(x_coords-3000, y_coords-3000, c=values, s=50, cmap='viridis')  # s controls size
plt.colorbar(scatter, label='Delta Phase')
plt.gca().invert_yaxis()  # To match imshow orientation if needed
plt.title('Delta Phase Scatter Plot')
plt.xlabel('Delta Q')
plt.ylabel('Delta I')

valid_points = ~np.isnan(heatmap_data_fast)
y_coords, x_coords = np.where(valid_points)
values = heatmap_data_fast[valid_points]

plt.figure(figsize=(12,10))
scatter = plt.scatter(x_coords[values > 0]-3000, y_coords[values > 0]-3000, c=values[values>0], s=50, cmap='viridis')  # s controls size
plt.colorbar(scatter, label='Delta Phase')
plt.gca().invert_yaxis()  # To match imshow orientation if needed
plt.title('Delta Phase Scatter Plot')
plt.xlabel('Delta Q')
plt.ylabel('Delta I')

plt.show()

In [ ]:
valid_points = ~np.isnan(heatmap_data)
y_coords, x_coords = np.where(valid_points)
values = heatmap_data[valid_points]

plt.figure(figsize=(12,10))
scatter = plt.scatter(x_coords-3000, y_coords-3000, c=values, s=2, cmap='viridis')  # s controls size
plt.colorbar(scatter, label='Delta Phase')
plt.gca().invert_yaxis()  # To match imshow orientation if needed
plt.title('Delta Phase Scatter Plot')
plt.xlabel('Delta Q')
plt.ylabel('Delta I')

valid_points = ~np.isnan(heatmap_data_fast)
y_coords, x_coords = np.where(valid_points)
values = heatmap_data_fast[valid_points]

plt.figure(figsize=(12,10))
scatter = plt.scatter(x_coords[values > 0]-3000, y_coords[values > 0]-3000, c=values[values>0], s=2, cmap='viridis')  # s controls size
plt.colorbar(scatter, label='Delta Phase')
plt.gca().invert_yaxis()  # To match imshow orientation if needed
plt.title('Delta Phase Scatter Plot')
plt.xlabel('Delta Q')
plt.ylabel('Delta I')

plt.show()

In [ ]:
#From here on out the hardware has been changed. Be warned!!
from qick import *
from qick.asm_v2 import *
%matplotlib inline
import matplotlib.pyplot as plt
#This line is to sync to an external clock which needs to be 10 Mhz
#soc = QickSoc(external_clk=True)
soc = QickSoc(bitfile ='/home/xilinx/jupyter_notebooks/qick/qick_lib/qick/d_1.bit', clk_output=True)
soccfg = soc
print(soccfg)


In [ ]:
# attempting to sweep a program for several cycles and achieve a continuous signal:
GEN_CH=0 # DAC 
RO_CH=0  # ADC

class SweepProgram(AveragerProgramV2):
    def _initialize(self, cfg):

        frequency = QickSweep1D("loopedy_loop", 100, 200)    ####HERE
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        self.declare_gen(ch=gen_ch, nqz=cfg['nqz'], ro_ch=ro_ch)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])
        self.add_readoutconfig(ch=ro_ch, name="myro",
                               freq=cfg['adc_freq'],
                               gen_ch=gen_ch,
                               outsel='product')
        self.add_pulse(ch=gen_ch, name="mypulse", ro_ch=ro_ch,
                       style="const",
#                        style="flat_top", 
#                        envelope="ramp", 
                       freq=frequency,                      ###HERE
                       length=cfg['flat_len'],
                       phase=cfg['phase'],
                       gain=cfg['gain'],
                      )
        self.add_reg("my_reg")
        self.add_loop("loopedy_loop", 100000, exec_before=None, exec_after=None)
        
        
    def _body(self, cfg):
        reg = QickRegisterV2("my_reg")
        self.pulse(ch=cfg['gen_ch'], name="mypulse", t=0.0)
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'])
        self.read_input(ro_ch = cfg['ro_ch'])
        self.write_dmem("r1", 's_port_h')
        self.write_dmem("r2", 's_port_l')
        reg.read_dmem("r1")
        #for channel 0 the tproc saves it in tproc input 0
        

        
config = {'gen_ch': GEN_CH,
          'ro_ch': RO_CH,
          'dac_freq': 110,
          'adc_freq': 110,
          'nqz': 1,
          'trig_time': 0.5,
          'ro_len': 0.004,
          'down_time':0.0059,
          'flat_len': .4,
          'ramp_len': 0.01,
          'phase': .875,
          'gain': 1.0
         }

prog = SweepProgram(soccfg, reps=1, final_delay=0.0, cfg=config)
print(prog)

#external trigger is currently available for tproc_2, on io pin 5, remove start_src if you do not want the trigger
prog.run(soc, start_src='external')
print("wmem:")
print(soccfg.read_mem(mem_sel = "wmem", length=1, addr=0))
print(soccfg.read_mem(mem_sel = "wmem", length=1, addr=1))
print(soccfg.read_mem(mem_sel = "wmem", length=1, addr=2))
print("dmem:")
print(soccfg.read_mem(mem_sel = "dmem", length=1, addr=0))
print(soccfg.read_mem(mem_sel = "dmem", length=1, addr=1))
print(soccfg.read_mem(mem_sel = "dmem", length=1, addr=2))